In [2]:
"""
Read before using:
DO NOT USE THIS SCRIPT AS A HACK SUCH AS AIMBOT OR AUTOAIM STUFF
This doesn't work since WT has cool methods to avoid this kind of actions
Anyway, since I think it's not a bad project, I didn't want to trash this 
so I decided to post this on github.
"""

from PIL import ImageGrab
import numpy as np
import cv2
import pyautogui
import keyboard
from IPython.display import clear_output
import win32api
from ahk import AHK


def useless(a):
    """
    function required by trackbars, it does nothing
    """
    pass

def get_mask_values():
    """
    get trackbars' values for to create the mask
    
    the mask should only get the contour of the plane and where you should aim to hit it
    """
    h_min = cv2.getTrackbarPos("Hue Min","TrackBars")
    h_max = cv2.getTrackbarPos("Hue Max", "TrackBars")
    s_min = cv2.getTrackbarPos("Sat Min", "TrackBars")
    s_max = cv2.getTrackbarPos("Sat Max", "TrackBars")
    v_min = cv2.getTrackbarPos("Val Min", "TrackBars")
    v_max = cv2.getTrackbarPos("Val Max", "TrackBars")
    lower = np.array([h_min,s_min,v_min])
    upper = np.array([h_max,s_max,v_max])
    
    return lower,upper

    
def thresholding(frame, lower, upper):
    mask = cv2.inRange(frame, lower, upper)
    res = cv2.bitwise_and(frame,frame, mask = mask)

    return mask,res


def zoom(img): 
    pts1 = np.float32([[100,100],[340,100],[100,200],[340,200]])
    pts2 = np.float32([[0,0],[440,0],[0,300],[440,300]])

    M = cv2.getPerspectiveTransform(pts1,pts2)
    zoomed = cv2.warpPerspective(img,M,(440,300))

    return zoomed


def getHist(img, minPer = 0.1, region = 1):
    """
    -based on which and how many pixel are filtered by the mask,
    the function creates 2 window with 2 different histograms (one for vertical pixel and one for horizontal ones)
    -it also add a circle on res window exactly where the plane is
    -returns circle coordinates and histograms np arrays 
    """
    if region == 1:
        histValX = np.sum(img, axis = 0)
        histValY = np.sum(img, axis = 1)
    else:
        histValX = np.sum(img[img.shape[0]//region:,:], axis = 0)
        histValY = np.sum(img[img.shape[1]//region:,:], axis = 1)
    
    maxValX = np.max(histValX)    
    maxValY = np.max(histValY)
    minValX = minPer * maxValX
    minValY = minPer * maxValY
    
    indexArrayX = np.where(histValX >= minValX)
    indexArrayY = np.where(histValY >= minValY) 
    cordX = np.average(indexArrayX)
    cordY = np.average(indexArrayY)
    
    histX = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
    histY = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
    
    for x,intensity in enumerate(histValX):
        if intensity >= 300:
            cv2.line(histX,(x, img.shape[1]), (x,img.shape[0]-intensity//255//region) , (255,0,0), 1)
    for y,intensity in enumerate(histValY):
        if intensity >= 300:
            cv2.line(histY,(0, y), (intensity//255//region-1,y), (0,0,255), 1)
    cv2.circle(res, (int(cordX), int(cordY)), 30, (0,0,255), 1, cv2.LINE_AA)
            
    #cv2.circle(hist, (int(cordX),int(cordY)), 25, (0,80,220), cv2.FILLED)
    return histX, histY, cordX, cordY

ahk = AHK() #creating an autohotkey object

#creating a window containg all the trackbars
cv2.namedWindow("TrackBars") 
cv2.resizeWindow("TrackBars",350,250)
cv2.createTrackbar("Hue Min","TrackBars",90,255,useless)
cv2.createTrackbar("Hue Max","TrackBars",108,255,useless)
cv2.createTrackbar("Sat Min","TrackBars",69,255,useless)
cv2.createTrackbar("Sat Max","TrackBars",90,255,useless)
cv2.createTrackbar("Val Min","TrackBars",29,255,useless)
cv2.createTrackbar("Val Max","TrackBars",73,255,useless)

while True:
    #grabbing the image(screen recording and changing colors)
    frame = np.array(ImageGrab.grab(bbox=(1100,500,1540,800)))
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    frame_hls = cv2.cvtColor(frame, cv2.COLOR_RGB2HLS)
    frame_luv = cv2.cvtColor(frame, cv2.COLOR_BGR2LUV)
    zoomed = zoom(frame_hls) #zooming
    
    lower, upper = get_mask_values() #trackbars values
    mask, res = thresholding(zoomed, lower, upper) #masking the zoomed frame to get only a frame with plane's boundingbox
    histX, histY, cordX, cordY = getHist(mask) #applying the mask and histograms
    
    #showing different frames and results
    #cv2.imshow("frame", frame)
    cv2.imshow("res", res)
    cv2.imshow("zoom", zoomed)
    cv2.imshow("histX", histX)
    cv2.imshow("histY", histY)
    
        
    k = cv2.waitKey(1)
    #clear_output()
    if keyboard.is_pressed("u"): #press u to track the plane
        ahk.mouse_move(cordX, cordY)
        #print(f"mouse moved @{cordX, cordY}")
    elif k == 113: #press k to stop the whole script
        break
        
cv2.destroyAllWindows()